In [91]:
import numpy as np
import math
import cv2
import utils
#import matplotlib.pyplot as plt

load image and generate binary image

In [92]:
image = utils.loadImage(path="./../src/testImages",name="raf2.png")
(thresh, binary) = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)


kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

utils.saveImage(name="image.png", image=image)
utils.saveImage(name="binary.png", image=binary)

filter functions

In [93]:
def getRegion(size, pos, image):
    xOffset = math.floor(size[0]/2)
    yOffset = math.floor(size[1]/2)
    return image[pos[0]-xOffset:pos[0]+xOffset+1, pos[1]-yOffset:pos[1]+yOffset+1]

def equal(filter, img):
    for x in range(0, len(filter)):
        for y in range(0, len(filter[0])):
            if filter[x][y] != -1 and filter[x][y] != img[x][y]:
                return False
    return True

def applyFilterOnPoints(filter, image, points):
    size = (len(filter),len(filter[0]))
    features = []
    for (x, y) in points:
        try:
            if equal(filter, getRegion(size, (x, y), image)):
                features.append((x,y))
        except:
            pass
    return features

def rotateFilter(filter):
    rotatedFilters = []
    for k in range(0,4):
        rotatedFilters.append(np.rot90(filter, k))
    return rotatedFilters

get Black pixel

In [94]:
points = []
for x in range(0, len(binary)):
    for y in range(0, len(binary[x])):
        if binary[x][y] == 255:
            points.append((x,y))

remove black sections

In [95]:
filter = np.full((13, 13), 255)
features = applyFilterOnPoints(filter, binary, points)

for (x,y) in features:
    binary[x][y] = 0

utils.saveImage(name="ohneSpulen.png", image=binary)

thinn image

In [96]:
thinn = utils.thinnImage(binary)

utils.saveImage(name="thinn.png", image=thinn)

replace certain features

In [97]:
cross = [[0,255,0],[255,255,255],[0,255,0]]
tcross = rotateFilter([[-1,0,-1],[255,255,255],[0,255,0]])
corner = rotateFilter([[0,-1,-1],[-1,255,255],[-1,255,0]])

crossFeatures = applyFilterOnPoints(cross, thinn, points)

for (x,y) in crossFeatures:
    thinn[x][y] = 0
    thinn[x][y-1] = 0
    thinn[x-1][y-1] = 255

tcrossFeatures = []
for filter in tcross:
    tcrossFeatures = applyFilterOnPoints(filter, thinn, points)
    for (x,y) in tcrossFeatures:
        thinn[x][y] = 0


for filter in corner:
    cornerFeatures = applyFilterOnPoints(filter, thinn, points)
    for (x,y) in cornerFeatures:
        thinn[x][y] = 0



utils.saveImage(name="modifiedThinn.png", image=thinn)